In [1]:

import os
import time
import oss2
import json

import cv2 as cv
import numpy as np
from tqdm import tqdm

from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdkfacebody.request.v20191230.DetectFaceRequest import DetectFaceRequest


In [2]:

access_key_id = os.getenv('OSS_TEST_ACCESS_KEY_ID', '')
access_key_secret = os.getenv('OSS_TEST_ACCESS_KEY_SECRET', '')
bucket_name = os.getenv('OSS_TEST_BUCKET', 'ivan-bucket-out-002')
endpoint = os.getenv('OSS_TEST_ENDPOINT', 'oss-cn-shanghai.aliyuncs.com')

bucket = oss2.Bucket(oss2.Auth(access_key_id, access_key_secret), endpoint, bucket_name)

client = AcsClient(access_key_id, access_key_secret, 'cn-shanghai')


In [3]:

#
def _score(img, face=None, isresponse=None):
    time.sleep(0.5)
    oss2.resumable_upload(bucket, 'demo.jpg', img)
    request = DetectFaceRequest()
    request.set_accept_format('json')

    request.set_ImageURL(f"https://{bucket_name}.{endpoint}/demo.jpg")

    response = client.do_action_with_exception(request)
    
    if isresponse:
        if "FaceRectangles" in json.loads(response)["Data"]:
            return json.loads(response)["Data"]["FaceRectangles"]
        else:
            return []
    if face:
        if "FaceRectangles" in json.loads(response)["Data"]:
            return json.loads(response)["Data"]["FaceRectangles"] != face
        else:
            return True
    return json.loads(response)["Data"]["FaceCount"] == 0


In [4]:

#
def f_lf1(img1, img2):
    img1, img2 = cv.imread(img1), cv.imread(img2)
    return np.sqrt(np.sum(np.square(img1 - img2)))


In [5]:
%time _score('../Data/Z0.jpg')

CPU times: user 9.15 ms, sys: 2.69 ms, total: 11.8 ms
Wall time: 814 ms


False

In [6]:
%time _score('../Data/Z0.jpg', isresponse=1)

CPU times: user 5.26 ms, sys: 1.57 ms, total: 6.83 ms
Wall time: 726 ms


[19, 22, 77, 89]

In [7]:
%time _score('../Data/Z0.jpg', face=[19, 22, 77, 89])

CPU times: user 4.57 ms, sys: 1.54 ms, total: 6.1 ms
Wall time: 743 ms


False

In [8]:
%time _score('../Data/bad.png')

CPU times: user 5.11 ms, sys: 2.08 ms, total: 7.18 ms
Wall time: 919 ms


True

In [9]:
%time _score('../Data/bad.png', isresponse=1)

CPU times: user 5.05 ms, sys: 1.45 ms, total: 6.5 ms
Wall time: 729 ms


[]

In [10]:

pNum = "T000001"

proj = ".."
pimg = f"{proj}/Data/images"
pmdl = f"{proj}/Model"
pout = f"{proj}/Out/{pNum}"

os.system(f"rm -rf {pout}/temp*")
os.system(f"rm -rf {pout}/images/*")


0

In [11]:

st, ed = 1, 1
chose = 1

_N = 128
lrun = [2, 4, 8, 16, 32, 64, 128][:chose]
pl = lambda x: sorted(
    [
        (i - _N // x, i, j - _N // x, j)
        for j in range(0, _N + 1, _N // x) if j > 0
        for i in range(0, _N + 1, _N // x) if i > 0
     ]
)
lrun


[2]

In [12]:

badl = []
group = 0

for i in tqdm(range(st, ed+1)):
    mscore = 999999
    pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
    op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    
    _face = _score(pi, isresponse=1)
    if not _face:
        continue
    
    for dN in tqdm([_i_ for _i_ in range(-25, 25+1) if _i_ != 0]):
        for _p_, M in enumerate(lrun):
            dpl = pl(M)

            if _p_ == 0:
                ldata = np.array([1 for _ in range(M**2)])
            else:
                t_ldata = [__i for __i, __j in zip(pl(lrun[_p_-1]), ldata) if __j == 1]
                ldata = []
                for _i in dpl:
                    _t = 0
                    for __i in t_ldata:
                        if _i[0] < __i[1] and __i[0] < _i[1] and _i[2] < __i[3] and __i[2] < _i[3]:
                            _t = 1
                            break
                    ldata.append(_t)
                ldata = np.array(ldata)

            #
            while True:
                t_ldata = ldata.copy()
                for _i in range(M**2):
                    if ldata[_i] == 1:
                        ldata[_i] = 0

                        img = cv.imread(pi)
                        for nldata, _ldata in enumerate(ldata):
                            if _ldata == 1:
                                i_group = dpl[nldata]
                                img[i_group[0]:i_group[1], i_group[2]:i_group[3]] = \
                                img[i_group[0]:i_group[1], i_group[2]:i_group[3]] - dN
                        cv.imwrite(f"{pout}/temp{group}.jpg", img)

                        escore = _score(f"{pout}/temp{group}.jpg", _face)
                        if escore:
                            pass
                        else:
                            ldata[_i] = 1
                if (ldata == t_ldata).all():
                    break
                break

        #
        img = cv.imread(pi)
        for nldata, _ldata in enumerate(ldata):
            if _ldata == 1:
                i_group = dpl[nldata]
                img[i_group[0]:i_group[1], i_group[2]:i_group[3]] = \
                img[i_group[0]:i_group[1], i_group[2]:i_group[3]] - dN
        cv.imwrite(f"{pout}/temp{group}.jpg", img)

        escore = _score(f"{pout}/temp{group}.jpg", _face)
        fscore = f_lf1(pi, f"{pout}/temp{group}.jpg")

        if escore:
            if fscore < mscore:
                print("\n", i, dN, escore, "%9.4f"%fscore)
                mscore = fscore
                os.system(f"cp {pout}/temp{group}.jpg {op}")



  2%|▏         | 1/50 [00:03<02:59,  3.67s/it]


 1 -25 True  906.6956



  4%|▍         | 2/50 [00:07<02:56,  3.67s/it]


 1 -24 True  846.6434



  6%|▌         | 3/50 [00:10<02:51,  3.66s/it]

...


 16%|█▌        | 8/50 [00:29<02:36,  3.72s/it]


 1 -18 True  756.9531



 56%|█████▌    | 28/50 [01:42<01:20,  3.65s/it]


 1 3 True  547.9544



100%|██████████| 1/1 [03:04<00:00, 184.87s/it]


In [13]:
st, el = 1, 712

savl = []
for i in tqdm(range(st, el+1)):
    pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
    op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    
    if not os.path.exists(op):
        img = cv.imread(pi)
        cv.imwrite(op, img)
        savl.append(1)
print(len(savl))


100%|██████████| 712/712 [00:00<00:00, 1173.25it/s]

711
